In [2]:
import torch

In [3]:
print(torch.cuda.is_available())

True


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader

In [5]:
# Define the paths to your raw and augmented data directories
raw_data_dir = r'C:\Users\user\Desktop\Minor sem 6\Laptop Components Image Dataset to Classify Different Components\Raw Data\Raw Data'  # Replace with the actual path to your raw data directory
augmented_data_dir = r'C:\Users\user\Desktop\Minor sem 6\Laptop Components Image Dataset to Classify Different Components\Augmented Data\Augmented Data'  # Replace with the actual path to your augmented data directory

In [6]:
# Define transformations for training and validation data
train_transforms = transforms.Compose([
    transforms.Resize((900, 1200)),  # Resize images
    transforms.RandomRotation(30),  # Random rotations for augmentation
    transforms.RandomHorizontalFlip(),  # Random horizontal flips
    transforms.RandomVerticalFlip(),  # Random vertical flips
    transforms.ToTensor(),  # Convert images to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization (ImageNet stats)
])

validation_transforms = transforms.Compose([
    transforms.Resize((900, 1200)),  # Resize images
    transforms.ToTensor(),  # Convert images to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization
])

# Load data using ImageFolder and DataLoader
train_data = datasets.ImageFolder(root=augmented_data_dir, transform=train_transforms)
valid_data = datasets.ImageFolder(root=raw_data_dir, transform=validation_transforms)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers = 4)
valid_loader = DataLoader(valid_data, batch_size=32, shuffle= True, num_workers = 4)

In [7]:
# Load the pre-trained ResNet model
model = models.resnet18(pretrained=True)

# Freeze all layers in the base model
for param in model.parameters():
    param.requires_grad = False

# Modify the fully connected (FC) layer to match our 26 categories
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 26)  # Output layer for 26 categories
)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

D:\hehe\envs\pytorch2\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\hehe\envs\pytorch2\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
D:\hehe\envs\pytorch2\Lib\site-packages\torch\nn\modules\module.py:1160: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\cb\pytorch_1000000000000\work\c10/cuda/CUDAAllocatorConfig.h:28.)
  return t.to(


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)  # Optimize only the final layers

In [9]:
torch.cuda.empty_cache()

In [1]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [10]:
from torch.cuda.amp import autocast, GradScaler

# Initialize the GradScaler for mixed precision
scaler = GradScaler()

# Number of epochs for training
num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    running_corrects = 0

    # Training phase
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Use autocast for mixed precision
        with autocast():  # Enable autocasting for automatic mixed precision
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        # Backward pass and optimization with scaled gradients
        scaler.scale(loss).backward()  # Scales the loss to prevent underflow in gradients
        scaler.step(optimizer)  # Optimizer step using the scaled gradients
        scaler.update()  # Update the scaler for the next iteration

        # Calculate statistics
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    # Calculate and print training statistics
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = running_corrects.double() / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_acc:.4f}')
    
    # Validation phase
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    val_corrects = 0

    # We don't need gradients for validation
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass with autocast for mixed precision
            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            # Calculate statistics
            _, preds = torch.max(outputs, 1)
            val_loss += loss.item() * inputs.size(0)
            val_corrects += torch.sum(preds == labels.data)

    # Calculate and print validation statistics
    val_loss = val_loss / len(valid_loader.dataset)
    val_acc = val_corrects.double() / len(valid_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}')


C:\Users\user\AppData\Local\Temp\ipykernel_8420\3291974304.py:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
C:\Users\user\AppData\Local\Temp\ipykernel_8420\3291974304.py:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Enable autocasting for automatic mixed precision


Epoch 1/10, Training Loss: 2.9204, Training Accuracy: 0.1163


C:\Users\user\AppData\Local\Temp\ipykernel_8420\3291974304.py:54: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/10, Validation Loss: 2.8308, Validation Accuracy: 0.1382
Epoch 2/10, Training Loss: 2.5829, Training Accuracy: 0.1841
Epoch 2/10, Validation Loss: 2.7488, Validation Accuracy: 0.1665
Epoch 3/10, Training Loss: 2.4357, Training Accuracy: 0.2186
Epoch 3/10, Validation Loss: 2.8851, Validation Accuracy: 0.1593
Epoch 4/10, Training Loss: 2.3331, Training Accuracy: 0.2491
Epoch 4/10, Validation Loss: 2.7645, Validation Accuracy: 0.1742
Epoch 5/10, Training Loss: 2.2696, Training Accuracy: 0.2659
Epoch 5/10, Validation Loss: 2.8222, Validation Accuracy: 0.1731
Epoch 6/10, Training Loss: 2.2053, Training Accuracy: 0.2838
Epoch 6/10, Validation Loss: 2.4734, Validation Accuracy: 0.2253
Epoch 7/10, Training Loss: 2.1599, Training Accuracy: 0.2921
Epoch 7/10, Validation Loss: 2.6247, Validation Accuracy: 0.2060
Epoch 8/10, Training Loss: 2.1417, Training Accuracy: 0.3018
Epoch 8/10, Validation Loss: 2.5578, Validation Accuracy: 0.2214
Epoch 9/10, Training Loss: 2.0898, Training Accuracy:

In [11]:
# Save the model's state_dict
torch.save(model.state_dict(), 'model_checkpoint.pth')